In [125]:
from abc import ABC, abstractmethod

In [269]:
class ConformError(BaseException):
    def __init__(self, reason=None):
        self.reason = reason


class SpecBase(ABC):
    def __init__(self, name):
        self.name = name
    
    @abstractmethod
    def conform(self, data):
        pass
    
    def is_valid(self, data):
        try:
            return True if self.conform(data) else False
        except ConformError(e):
            return False
        except:
            raise
        
    
class SimpleSpec(SpecBase):
    def __init__(self, predicate, name=None):
        super().__init__(name if name else 'spec<{}>'.format(predicate.__name__))   
        self.predicate = predicate
        
    def conform(self, data):
        result = self.predicate(data)
        if result:
            return (True, data, result)
        else:
            raise ConformError((self.name, self.predicate, data, result))

In [156]:
def is_spec_or_predicate(x):
    import collections
    
    return isinstance(x, collections.Callable) or isinstance(x, SpecBase)


def is_spec(x):
    return isinstance(x, SpecBase)


def spec(predicate, name=None):
    from copy import deepcopy
    
    assert is_spec_or_predicate(predicate)
    
    if is_spec(predicate):
        new_spec = deepcopy(predicate)
        new_spec.name = name if name else predicate.name
        return new_spec
    
    return SimpleSpec(predicate, name)

In [157]:
integer = spec(lambda x: isinstance(x, int), 'integer?')

In [158]:
integer.is_valid(100)

True

In [159]:
integer.is_valid('a')

False

In [160]:
def is_valid(spc, data):
    assert is_spec(spc)
    
    return spc.is_valid(data)

In [161]:
def conform(spc, data):
    assert is_spec(spc)
    
    return spc.conform(data)

In [162]:
conform(integer, 100)

(True, 100, True)

In [163]:
conform(integer, None)

ConformError: ('integer?', <function <lambda> at 0x10df24730>, None, False)

In [219]:
class AndSpec(SpecBase):
    def __init__(self, specs, name=None):
        self.specs = specs
        for s in self.specs:
            assert is_spec(s)
        self.name = 'AndSpec' if not name else name
        
    def conform(self, data):
        conf_result = None
        
        for s in self.specs:
            conf_result = s.conform(data)
            
        return conf_result
     
    
class OrSpec(SpecBase):
    def __init__(self, specs, name=None):
        self.specs = specs
        for s in self.specs:
            assert is_spec(s)
        self.name = 'OrSpec' if not name else name
        
    def conform(self, data):
        conformed = False
        conf_result = None
        
        for s in self.specs:
            if isinstance(s, SpecBase):
                try:
                    conf_result = s.conform(data)
                    conformed = True
                except:
                    conformed = False
            
            if conformed:
                return conf_result
        
        raise ConformError((self.name, self.specs, data, None))

In [220]:
def and_spec(*spc, **kargs):
    return AndSpec(spc, **kargs)


def or_spec(*spc, **kargs):
    return OrSpec(spc, **kargs)

In [221]:
string = spec(lambda x: isinstance(x, str), 'string?')

In [222]:
string10 = and_spec(string, spec(lambda x: len(x) < 10, 'len<10?'), name='string_with_len_10?')

In [223]:
conform(string10, 'hola')

(True, 'hola', True)

In [224]:
is_valid(string10, 11)

False

In [225]:
is_valid(string10, 'xxxxxxxxzxxcxcxcxcxccxcxccxcxcx')

False

In [226]:
sp3 = or_spec(integer, string10)

In [227]:
is_valid(sp3, 1)

True

In [228]:
is_valid(sp3, 'aaaaa')

True

In [229]:
is_valid(sp3, 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdffds')

False

In [230]:
conform(sp3, 3.4)

ConformError: ('OrSpec', (<__main__.SimpleSpec object at 0x10df30208>, <__main__.AndSpec object at 0x10df73400>), 3.4, None)

In [231]:
conform(string10, 90)

ConformError: ('string?', <function <lambda> at 0x10ddec510>, 90, False)

In [265]:
class DictSpec(SpecBase):
    def __init__(self, req=None, opt=None, allow_more=True, name=None):
        self.name = 'DictSpec' if not name else name
        self.req_specs = dict() if not req else req
        self.opt_specs = dict() if not opt else opt
        self.allow_more = allow_more
        
    def conform(self, data):
        if not isinstance(data, dict):
            raise ConformError((self.name, dict, data, None))
            
        for rk in self.req_specs.keys():
            assert is_spec(self.req_specs[rk])
            
            val = data.get(rk, None)
            if val is None:
                raise ConformError(('required_key {}'.format(rk), self.req_specs[rk], data, None))
                
            self.req_specs[rk].conform(val)
        
        for rk in self.opt_specs.keys():
            assert is_spec(self.opt_specs[rk])
            
            val = data.get(rk, None)
            
            if val:
                self.opt_specs[rk].conform(val)
            
        if not self.allow_more:
            total_keys = set(list(self.req_specs.keys()) + list(self.opt_specs.keys()))
            all_keys = set(data.keys())
            
            rem_key_count = len(all_keys - total_keys)
            
            if rem_key_count > 0:
                raise ConformError(('more_data', self, data, None))
                
        return (True, data, None)
    

def dict_spec(req=None, opt=None, name=None, allow_more=True):
    return DictSpec(req=req, opt=opt, name=name, allow_more=allow_more)

In [254]:
first_name = spec(string10, 'first_name')

In [255]:
last_name = and_spec(string, spec(lambda x: len(x) <= 35, 'len<35'), name='last_name')

In [256]:
age = spec(integer)

In [257]:
person = dict_spec(req=dict(fname=first_name, lname=last_name), opt=dict(age=age))

In [258]:
conform(person, {'fname': 'Sourav', 'lname': 'Datta'})

(True, {'fname': 'Sourav', 'lname': 'Datta'}, None)

In [259]:
conform(person, {'fname': 'Sourav'})

ConformError: ('required_key lname', <__main__.AndSpec object at 0x10df62f60>, {'fname': 'Sourav'}, None)

In [260]:
conform(person, {'lname': 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsffdsfdsfsfdsfdsfdsfdsfsdfdsfdsfdsfdsffdsfdsfds'})

ConformError: ('required_key fname', <__main__.AndSpec object at 0x10df62b00>, {'lname': 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsffdsfdsfsfdsfdsfdsfdsfsdfdsfdsfdsfdsffdsfdsfds'}, None)

In [261]:
conform(person, {'fname': 'Ok', 'lname': 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsffdsfdsfsfdsfdsfdsfdsfsdfdsfdsfdsfdsffdsfdsfds'})

ConformError: ('len<35', <function <lambda> at 0x10df671e0>, 'sdfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsfdsffdsfdsfsfdsfdsfdsfdsfsdfdsfdsfdsfdsffdsfdsfds', False)

In [262]:
conform(person, {'fname': 'Sourav', 'lname': 'Datta', 'age': '35 years'})

ConformError: ('integer?', <function <lambda> at 0x10df24730>, '35 years', False)

In [266]:
person = dict_spec(req=dict(fname=first_name, lname=last_name), opt=dict(age=age), name='person', allow_more=False)

In [267]:
conform(person, {'fname': 'Sourav', 'lname': 'Datta', 'age': 35})

(True, {'age': 35, 'fname': 'Sourav', 'lname': 'Datta'}, None)

In [268]:
conform(person, {'fname': 'Sourav', 'lname': 'Datta', 'age': 35, 'address': 'Bristol'})

ConformError: ('more_data', <__main__.DictSpec object at 0x10df807b8>, {'fname': 'Sourav', 'lname': 'Datta', 'age': 35, 'address': 'Bristol'}, None)